<a href="https://colab.research.google.com/github/sakshimane7/Coding-Raja-Technologies-Internship/blob/main/movie_recommendations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import pandas as pd

# https://files.grouplens.org/datasets/movielens/ml-25m.zip

movies = pd.read_csv("movies.csv")





In [3]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [47]:
import re

def clean_title(title):
  title = re.sub("[^a-zA-Z0-9 ]", "", title)
  return title

In [46]:
movies["clean_title"] = movies["title"].apply(clean_title)

In [48]:
movies

,movieId,title,genres,clean_title
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,Toy Story 1995
1,2,Jumanji (1995),Adventure|Children|Fantasy,Jumanji 1995
2,3,Grumpier Old Men (1995),Comedy|Romance,Grumpier Old Men 1995
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,Waiting to Exhale 1995
4,5,Father of the Bride Part II (1995),Comedy,Father of the Bride Part II 1995
...,...,...,...,...
62418,209157,We (2018),Drama,We 2018
62419,209159,Window of the Soul (2001),Documentary,Window of the Soul 2001
62420,209163,Bad Poems (2018),Comedy|Drama,Bad Poems 2018
62421,209169,A Girl Thing (2001),(no genres listed),A Girl Thing 2001


In [51]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,2))

tfidf = vectorizer.fit_transform(movies["clean_title"])


In [12]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def search(title):
    title = clean_title(title)
    query_vec = vectorizer.transform([title])
    similarity = cosine_similarity(query_vec, tfidf).flatten()
    indices = np.argpartition(similarity, -5)[-5:]
    results = movies.iloc[indices].iloc[::-1]

    return results

In [13]:
# pip install ipywidgets
#jupyter labextension install @jupyter-widgets/jupyterlab-manager

In [14]:
import ipywidgets as widgets
from IPython.display import display

movie_input = widgets.Text(
    value='Toy Story',
    description='Movie Title:',
    disabled=False
)
movie_list = widgets.Output()

def on_type(data):
    with movie_list:
        movie_list.clear_output()
        title = data["new"]
        if len(title) > 5:
            display(search(title))

movie_input.observe(on_type, names='value')


display(movie_input, movie_list)

Text(value='Toy Story', description='Movie Title:')

Output()

In [15]:
movie_id = 89745

#def find_similar_movies(movie_id):
movie = movies[movies["movieId"] == movie_id]

In [16]:
ratings = pd.read_csv("ratings.csv")

In [18]:
ratings.dtypes

userId         int64
movieId        int64
rating       float64
timestamp    float64
dtype: object

In [20]:
similar_users = ratings[(ratings["movieId"] == movie_id) & (ratings["rating"] > 4)]["userId"].unique()

In [21]:
similar_user_recs = ratings[(ratings["userId"].isin(similar_users)) & (ratings["rating"] > 4)]["movieId"]

In [22]:
similar_user_recs = similar_user_recs.value_counts() / len(similar_users)

similar_user_recs = similar_user_recs[similar_user_recs > .10]

In [23]:
all_users = ratings[(ratings["movieId"].isin(similar_user_recs.index)) & (ratings["rating"] > 4)]

In [24]:
all_user_recs = all_users["movieId"].value_counts() / len(all_users["userId"].unique())

In [25]:
rec_percentages = pd.concat([similar_user_recs, all_user_recs], axis=1)
rec_percentages.columns = ["similar", "all"]

In [26]:
rec_percentages

,similar,all
89745,1.000000,0.040684
58559,0.564829,0.145880
59315,0.509687,0.055721
79132,0.509687,0.134299
112852,0.479881,0.044079
...,...,...
112552,0.104322,0.037531
5218,0.102832,0.023404
2115,0.102832,0.038137
4973,0.101341,0.110835


In [27]:
rec_percentages["score"] = rec_percentages["similar"] / rec_percentages["all"]

In [28]:
rec_percentages = rec_percentages.sort_values("score", ascending=False)

In [29]:
rec_percentages.head(10).merge(movies, left_index=True, right_on="movieId")

,similar,all,score,movieId,title,genres
17067,1.000000,0.040684,24.579732,89745,"Avengers, The (2012)",Action|Adventure|Sci-Fi|IMAX
25058,0.238450,0.012975,18.377370,122892,Avengers: Age of Ultron (2015),Action|Adventure|Sci-Fi
19678,0.198212,0.011278,17.575830,102125,Iron Man 3 (2013),Action|Sci-Fi|Thriller|IMAX
21348,0.272727,0.016128,16.910116,110102,Captain America: The Winter Soldier (2014),Action|Adventure|Sci-Fi|IMAX
16312,0.174367,0.010368,16.817711,86332,Thor (2011),Action|Adventure|Drama|Fantasy|IMAX
16725,0.184799,0.011217,16.475063,88140,Captain America: The First Avenger (2011),Action|Adventure|Sci-Fi|Thriller|War
25061,0.132638,0.008064,16.448091,122900,Ant-Man (2015),Action|Adventure|Sci-Fi
25071,0.220566,0.013460,16.386488,122920,Captain America: Civil War (2016),Action|Sci-Fi|Thriller
14628,0.229508,0.015158,15.141115,77561,Iron Man 2 (2010),Action|Adventure|Sci-Fi|Thriller|IMAX
21606,0.242921,0.016249,14.949613,111362,X-Men: Days of Future Past (2014),Action|Adventure|Sci-Fi


In [30]:
def find_similar_movies(movie_id):
    similar_users = ratings[(ratings["movieId"] == movie_id) & (ratings["rating"] > 4)]["userId"].unique()
    similar_user_recs = ratings[(ratings["userId"].isin(similar_users)) & (ratings["rating"] > 4)]["movieId"]
    similar_user_recs = similar_user_recs.value_counts() / len(similar_users)

    similar_user_recs = similar_user_recs[similar_user_recs > .10]
    all_users = ratings[(ratings["movieId"].isin(similar_user_recs.index)) & (ratings["rating"] > 4)]
    all_user_recs = all_users["movieId"].value_counts() / len(all_users["userId"].unique())
    rec_percentages = pd.concat([similar_user_recs, all_user_recs], axis=1)
    rec_percentages.columns = ["similar", "all"]

    rec_percentages["score"] = rec_percentages["similar"] / rec_percentages["all"]
    rec_percentages = rec_percentages.sort_values("score", ascending=False)
    return rec_percentages.head(10).merge(movies, left_index=True, right_on="movieId")[["score", "title", "genres"]]

In [31]:
import ipywidgets as widgets
from IPython.display import display

movie_name_input = widgets.Text(
    value='Toy Story',
    description='Movie Title:',
    disabled=False
)
recommendation_list = widgets.Output()

def on_type(data):
    with recommendation_list:
        recommendation_list.clear_output()
        title = data["new"]
        if len(title) > 5:
            results = search(title)
            movie_id = results.iloc[0]["movieId"]
            display(find_similar_movies(movie_id))

movie_name_input.observe(on_type, names='value')

display(movie_name_input, recommendation_list)

Text(value='Toy Story', description='Movie Title:')

Output()